In [1]:
! nvidia-smi

Tue Apr 25 18:11:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.78       Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8     7W /  N/A |    224MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
! pip install plotly -q

In [7]:
! git clone https://github.com/openai/point-e

Cloning into 'point-e'...


In [5]:
%cd point-e

[WinError 2] The system cannot find the file specified: 'point-e'
c:\Users\HP\OneDrive\Documents\GitHub\point_cloud\AI Converts Photo to 3D Point Cloud Object


In [4]:
! pip install -e . -q

ERROR: file:///C:/Users/HP/OneDrive/Documents/GitHub/point_cloud/AI%20Converts%20Photo%20to%203D%20Point%20Cloud%20Object does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [6]:
from PIL import Image
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud


ModuleNotFoundError: No module named 'point_e'

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base40M'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))


creating base model...


100%|███████████████████████████████████████| 890M/890M [04:24<00:00, 3.53MiB/s]


creating upsample model...
downloading base checkpoint...


100%|██████████| 162M/162M [00:55<00:00, 2.91MiB/s] 


downloading upsampler checkpoint...


100%|██████████| 162M/162M [00:40<00:00, 4.01MiB/s] 


<All keys matched successfully>

In [10]:
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 3.0],
)


In [11]:
from google.colab import files
uploaded = files.upload()


ModuleNotFoundError: No module named 'google.colab'

In [2]:
# Load an image to condition on.
img = Image.open("C:\\Users\\HP\\OneDrive\\Desktop\\png-clipart-table-wood-furniture-table-angle-rectangle-thumbnail.png")
#a_chair(1).png
# Produce a sample from the model.
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=[img]))):
    samples = x


NameError: name 'Image' is not defined

In [ ]:
img


In [ ]:
pc = sampler.output_to_point_clouds(samples)[0]


In [ ]:
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

In [ ]:
import plotly.graph_objects as go


In [ ]:
fig_plotly = go.Figure(
    data=[
        go.Scatter3d(
            x=pc.coords[:, 0], y=pc.coords[:, 1], z=pc.coords[:, 2],
            mode='markers',
            marker=dict(
                size=2,
                color=['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(
                    pc.channels["R"], pc.channels["G"], pc.channels["B"])],
            )
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    ),
)


In [ ]:
fig_plotly.show(renderer="colab")


In [ ]:
from point_e.util.pc_to_mesh import marching_cubes_mesh


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating SDF model...')
name = 'sdf'
model = model_from_config(MODEL_CONFIGS[name], device)
model.eval()

print('loading SDF model...')
model.load_state_dict(load_checkpoint(name, device))


In [ ]:
import skimage.measure as measure


In [ ]:
# Produce a mesh (with vertex colors)
mesh = marching_cubes_mesh(
    pc=pc,
    model=model,
    batch_size=4096,
    grid_size=32,  # increase to 128 for resolution used in evals
    progress=True,
)


In [ ]:
# Write the mesh to a PLY file to import into some other program.
with open('chair.ply', 'wb') as f:
    mesh.write_ply(f)
